# Progetto PAA, a.a. 2015/2016

Autori: Tommaso Papini (5537529), Gabriele Bani (5719258)

## Quicksort


In [525]:
import sympy
import numpy
from sympy import *
from sympy.abc import x, n, z, t

init_printing() # for nice printing, a-la' TeX

In [526]:
def contaquick(v,l,r):
    #v is the array, l and r are the indexes of the quicksort (left and right)
    p = Symbol('p')
    i = Symbol('i')
    j = Symbol('j')
    a = Symbol('a')
    nc = Symbol('nc') #nc is the number of checks made by the QuickSort Algorithm
    nc = 0;
    if r-l>=0:
        p = v[r]
        i = l-1
        j = r
        while i<j:
            i=i+1
            nc=nc+1
            while v[i]<p:
                i=i+1
                nc=nc+1
            j=j-1
            nc=nc+1
            while l<=j and p<v[j]:
                j=j-1
                nc=nc+1
            if i<j:
                a=v[i]
                v[i]=v[j]
                v[j]=a
        a=v[i]
        v[i]=p
        v[r]=a
        nc = nc + contaquick(v,l,i-1)
        nc = nc + contaquick(v,i+1,r)
    return nc

In [527]:
def checks_quicksort(n):
    return 2*(n+1)*(harmonic(n+1).evalf()-1)

In [528]:
permutazione_random = [12,11,9,13,15,6,3,20,17,14,4,19,7,2,1,10,5,18,8,16]
permutazione_random

In [529]:
contaquick(permutazione_random,0,19)

In [530]:
permutazione_random

In [531]:
permutazione_random = [13,12,7,4,10,5,17,15,18,19,1,2,14,20,11,8,6,3,16,9]
permutazione_random

In [532]:
contaquick(permutazione_random,0,19)

In [533]:
permutazione_random

In [534]:
permutazione_random = [19,7,20,18,13,9,2,5,15,3,12,6,14,10,1,16,11,17,4,8]
permutazione_random

In [535]:
contaquick(permutazione_random,0,19)

In [536]:
permutazione_random

In [537]:
permutazione_random = numpy.random.permutation(20)
permutazione_random

array([11, 16,  9, 15,  7,  0,  8, 17, 14,  2,  6,  4,  5, 13, 18, 10, 19,
        1,  3, 12])

In [538]:
contaquick(permutazione_random,0,19)

In [539]:
permutazione_random

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [540]:
checks_quicksort(20)

In [541]:
permutazione_random = numpy.random.permutation(4)
permutazione_random

array([0, 3, 1, 2])

In [542]:
contaquick(permutazione_random,0,3)

In [543]:
permutazione_random

array([0, 1, 2, 3])

In [544]:
checks_quicksort(4)